In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from os import listdir
from os.path import isfile, join
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer

In [2]:
path="C:\\Users\\nilu1\\PatternCracking\\acc\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
temp.sort()
for i in range(len(temp)):
    st=temp[i]
    if(st[:2]=="18"):
        pathex.append(temp[i])
# print(pathex)

In [3]:
# n -> number of lines in csv file
# actual -> the actual key pressed
n=25
actual=1
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    dff=pd.read_csv(path+pathex[i])
    mini=10000
    minkeyd=1000
    minkeyu=1000
    keydi=pathex[i].rfind("_")+4
    keyui=pathex[i].rfind("&")+1
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keydi:keyui-1]))
        if(t < mini):
            mini=t
            minkeyd=j
    mini=10000
    for j in dff.index:
        t=abs(dff['Time'][j] - int(pathex[i][keyui:-4]))
        if(t < mini):
            mini=t
            minkeyu=j
#     print(minkeyd)
#     print(minkeyu)
    window=minkeyu-minkeyd
    left=n-window    
    dff=dff.head(minkeyu + round(left/2))
    dff=dff.tail(n)
    dff.reset_index(inplace=True)
    dff=dff.drop(columns="index")
    dff=dff.drop(columns="Time")
    df1=dff.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    dff=df1.to_frame().T
    ind=pathex[i].rfind("_")+3
#     print(pathex[i])
    dff.insert(n*3, "key", int(pathex[i][ind]))
    if(i==0):
        df=dff
    else:
        df=df.append(dff,ignore_index = True)
# df

In [4]:
X = df.iloc[:, 0:n*3].values
y = df.iloc[:, n*3].values

In [5]:
#divide train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [6]:
# sc = Normalizer()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [7]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred=y_pred.round()
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

,Actual,Predicted
0,9,3.0
1,6,6.0
2,7,6.0
3,6,6.0
4,1,5.0
5,2,-2.0


In [8]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
f1_score(y_test, y_pred, average="micro")

Mean Absolute Error: 2.5
Mean Squared Error: 11.5
Root Mean Squared Error: 3.391164991562634


0.3333333333333333